In [12]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [13]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [14]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [15]:
from HFCs_bmimBF4_PR import configuration

# Load data from csv
data = pd.read_csv('HFCSBF4_g.csv')

solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [16]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(250000)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.05)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.05)
    m.fs.state_block.mole_frac_comp["bmimBF4"].fix(0.90)
    
    m.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
    m.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
    m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].fix(-0.02825)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].fix(-0.00684)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].fix(0.57152)
    m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].fix(0.06453)
    
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp["R125"].unfix()
    m.fs.state_block.mole_frac_comp["R32"].unfix()
    m.fs.state_block.mole_frac_comp["bmimBF4"].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))   
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"])) 
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimBF4'].fix(float(data["x_IL"])) 
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['R32', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa_A['R32', 'R125'].setub(3)

    m.fs.properties.PR_kappa_A['R125', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa_A['R125', 'R32'].setub(3)

    m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].setlb(-3)
    m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].setub(3)

    m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].setub(3)
    
    m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].setub(3)

    m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].setlb(-3)
    m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].setub(3)
    
    # Return initialized flash model
    return m

In [17]:
import pytest

test_data = {"temperature": 298.15, "pressure": 250000, "x_R32":0.179, "x_IL":0.821,  
             "x_R125":0.00001, "y_R125":0.00001, "y_R32":0.9999, "y_IL":0.0001}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))


2022-01-26 15:46:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-01-26 15:46:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 1
The initial DOF is 1


In [18]:
variable_name = [
                "fs.properties.PR_kappa_A['R32', 'R125']",
                "fs.properties.PR_kappa_A['R125', 'R32']",
                "fs.properties.PR_kappa_A['R32', 'bmimBF4']",
                "fs.properties.PR_kappa_A['bmimBF4', 'R32']",
                "fs.properties.PR_kappa_A['bmimBF4', 'R125']",
                "fs.properties.PR_kappa_A['R125', 'bmimBF4']"]

In [19]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-4

# def SSE(m, data):
#     expr = ((float(data["y_R125"]) -
#              m.fs.state_block.mole_frac_phase_comp["Vap", "R125"])**2)


In [20]:
# solver_options = {'halt_on_ampl_error':'yes','tol': 1e-7, 'symbolic_solver_labels': True}
solver_options = {'halt_on_ampl_error':'yes'}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee = True, solver_options=solver_options, diagnostic_mode=True)
# solver_options = {'tol': 1e-6, 'max_iter':100}
# pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee = True)

obj_value, parameters = pest.theta_est()

2022-01-26 15:46:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-01-26 15:46:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-01-26 15:46:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-01-26 15:46:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-01-26 15:46:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-01-26 15:46:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
Ipopt 3.13.2: halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL)

  19r 7.8713381e+03 6.85e+00 9.99e+02   0.8 0.00e+00    -  0.00e+00 0.00e+00R  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20r 7.8712336e+03 4.72e+00 9.99e+02   0.8 6.84e+03    -  4.28e-04 9.93e-04f  1
  21r 7.8710723e+03 4.69e+00 9.95e+02   0.8 7.66e+02    -  3.68e-03 1.56e-03f  1
  22r 7.8705441e+03 4.67e+00 9.60e+02   0.8 1.24e+02    -  3.47e-02 5.88e-03f  1
  23r 7.8662087e+03 4.51e+00 1.51e+03   0.8 1.09e+02    -  8.15e-02 4.24e-02f  1
  24r 7.8509926e+03 4.20e+00 3.03e+03   0.8 9.99e+01    -  4.16e-01 1.20e-01f  1
  25r 7.8495878e+03 4.73e+00 4.23e+02   0.8 9.59e+00    -  3.79e-01 7.01e-01f  1
  26r 7.8468954e+03 4.17e+00 2.39e+03   0.8 2.62e+01    -  9.90e-01 1.00e+00f  1
  27r 7.8472143e+03 3.01e+00 5.08e+01   0.8 3.51e+01    -  7.45e-01 2.74e-01f  2
  28r 7.8511232e+03 1.97e+00 1.38e+02   0.8 9.98e+00    -  6.61e-01 1.00e+00f  1
  29  2.0411365e+03 7.57e-01 1.21e+01  -1.0 1.21e+04    -  5.94e-01 4.95e-01f  2
iter    objective    inf_pr 

In [21]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 0.000000

The values for the parameters are as follows:
fs.properties.PR_kappa_A[R32,R125] = 0.3916359477184864
fs.properties.PR_kappa_A[R125,R32] = 0.01069166646269672
fs.properties.PR_kappa_A[R32,bmimBF4] = -0.054073761238271414
fs.properties.PR_kappa_A[bmimBF4,R32] = 0.05802701882504397
fs.properties.PR_kappa_A[bmimBF4,R125] = 0.44167181942041706
fs.properties.PR_kappa_A[R125,bmimBF4] = -0.04024769911710136
The SSE at the optimal solution is 0.000000

The values for the parameters are as follows:
fs.properties.PR_kappa_A[R32,R125] = 0.3916359477184864
fs.properties.PR_kappa_A[R125,R32] = 0.01069166646269672
fs.properties.PR_kappa_A[R32,bmimBF4] = -0.054073761238271414
fs.properties.PR_kappa_A[bmimBF4,R32] = 0.05802701882504397
fs.properties.PR_kappa_A[bmimBF4,R125] = 0.44167181942041706
fs.properties.PR_kappa_A[R125,bmimBF4] = -0.04024769911710136


In [22]:
print(large_residuals_set(pest.ef_instance))

ComponentSet([])
ComponentSet([])
